In [22]:
from glob import glob
from datasets import Dataset
import numpy as np
import pandas as pd
from pprint import pprint
import os
import json
import g4f
from time import sleep

In [5]:
ds = Dataset.from_parquet("./data/train.parquet")
ds = ds.filter(lambda x: x["lang"] == "vi")
dolphin = ds.filter(lambda x: x["source"] == "dolphin")
orca_mqa = ds.filter(lambda x: x["source"] == "orca_mqa")

In [23]:
ds.to_pandas()['source'].value_counts() / len(ds)

alpaca      0.432246
dolphin     0.294713
orca_mqa    0.147356
wizard      0.098238
dolly       0.027448
Name: source, dtype: float64

In [104]:
TEMPLATE1 = """Sau đây là một phản hồi cho người dùng, hãy viết lại phản hồi thoả mãn các tiêu chí sau:
1) Nếu trong phản hồi đề cập tới bất kỳ nguồn thông tin nào để trả lời cho câu hỏi như: 'Trong đoạn văn được đề cập tới', 'Theo báo cáo', 'trong tài liệu', 'trong bài báo', 'trong đoạn trích', ... Hãy sửa nguồn thông tin đó thành 'nguồn tài liệu được cung cấp'.
2) Giữ nguyên những nguồn thông tin khác.
3) Sửa câu trả lời đề nó trở nên tự nhiên hơn.
***
Ví dụ:
- Phản hồi gốc: Sau khi đọc bài viết, câu trả lời đúng nhất là: C: không nên vứt túi đựng đồ ăn trưa. Bài báo đề cập rằng các câu lạc bộ môi trường yêu cầu học sinh mang bữa trưa của mình trong túi để có thể sử dụng lại, như một phần trong nỗ lực làm cho môi trường trong sạch.
- Phản hổi được viết lại: Dựa vào nguồn tài liệu được cung cấp, câu trả lời đúng nhất là: C: không nên vứt túi đựng đồ ăn trưa. Tài liệu đề cập rằng các câu lạc bộ môi trường yêu cầu học sinh mang bữa trưa của mình trong túi để có thể sử dụng lại, như một phần trong nỗ lực làm cho môi trường trong sạch.
***
- Phản hồi người dùng: {question}
"""
TEMPLATE2 = """Lưu ý: Chỉ trả về phản hồi được viết lại dưới dạng json {"rewritten_prompt": "your answer you rewrite"}, không cần giải thích gì thêm."""


def sample_pool(n, pool):
    if n >= len(pool):
        return pool
    return list(np.random.choice(pool, size=n, replace=False))


def sample_seed(n, seed):
    return list(np.random.choice(seed, size=n, replace=False))


def get_response(prompt):
    content = TEMPLATE1.format(question=prompt) + TEMPLATE2
    print(content)
    while True:
        try:
            response = g4f.ChatCompletion.create(
                provider=g4f.Provider.GeekGpt,
                model=g4f.models.gpt_4,
                messages=[{"role": "user", "content": content}],
            )
            return response
        except Exception as e:
            print(e)
            print("Request failed")
            sleep(10)


def extract_json(response):
    start = response.find("[")
    end = response.find("]") + 1
    if start == -1 or end == -1:
        return None
    try:
        return eval(response[start:end])
    except Exception as e:
        return None

In [106]:
temp = dolphin.shuffle()[0]
pprint(temp['output'])

('Bài viết nói về Kathryn Smith và cộng sự của cô là Matthew Rigby, những '
 'người đang bị xét xử vì tội giết con gái Ayeeshia Jane Smith của họ. '
 'Ayeeshia chết vì trái tim rách nát do một cú va chạm mạnh, và cả cha và mẹ '
 'đều phủ nhận cáo buộc. Bài báo thảo luận về tiền sử tự làm hại bản thân, lạm '
 'dụng chất gây nghiện và các vấn đề sức khỏe tâm thần của Kathryn Smith cũng '
 'như việc cô mang thai ngoài ý muốn. Nó cũng đề cập đến sự tham gia của các '
 'dịch vụ xã hội, những người đã phát hiện ra Smith sống trong điều kiện không '
 'phù hợp và sau đó lo ngại về khả năng chăm sóc con của cô ấy do các vấn đề '
 'cá nhân của cô ấy. Phiên tòa đang diễn ra, Smith và Rigby cũng không nhận '
 'tội cho phép hoặc gây ra cái chết và hành vi tàn ác đối với trẻ em.')


In [111]:
pprint(temp['instruction'])

('Bạn là một trợ lý AI. Người dùng sẽ giao cho bạn một nhiệm vụ. Mục tiêu của '
 'bạn là hoàn thành nhiệm vụ một cách trung thực nhất có thể. Trong khi thực '
 'hiện nhiệm vụ, hãy suy nghĩ từng bước một và biện minh cho các bước của '
 'bạn.\n'
 'Tòa án Birmingham Crown cho biết Kathryn Smith, 23 tuổi, có tiền sử tự làm '
 'hại bản thân, lạm dụng chất gây nghiện và các vấn đề về sức khỏe tâm thần và '
 'việc mang thai của cô không phải là kế hoạch. Ayeeshia Jane Smith qua đời '
 'tại nhà riêng ở Burton-upon-Trent, Staffordshire, khi trái tim cô bị xé nát '
 'bởi một cú va chạm mạnh, bồi thẩm đoàn cho biết. Cô Smith và bạn trai '
 'Matthew Rigby, 22 tuổi, đều đến từ Nottingham, phủ nhận tội giết người. Tòa '
 'án cho biết cô Smith đang mang thai Ayeeshia gần ba tháng khi nữ hộ sinh '
 'Hilary Sayles tìm thấy cô sống trong gara của cha cô ở Overseal, Derbyshire. '
 'Bà Sayles nói: "Có một chiếc giường và đồ nội thất nhưng rõ ràng là nó không '
 'phù hợp với em bé mới sinh - hoặc bất kỳ a

In [107]:
a = get_response(temp['output'])

Sau đây là một phản hồi cho người dùng, hãy viết lại phản hồi thoả mãn các tiêu chí sau:
1) Nếu trong phản hồi đề cập tới bất kỳ nguồn thông tin nào để trả lời cho câu hỏi như: 'Trong đoạn văn được đề cập tới', 'Theo báo cáo', 'trong tài liệu', 'trong bài báo', 'trong đoạn trích', ... Hãy sửa nguồn thông tin đó thành 'nguồn tài liệu được cung cấp'.
2) Giữ nguyên những nguồn thông tin khác.
3) Sửa câu trả lời đề nó trở nên tự nhiên hơn.
***
Ví dụ:
- Phản hồi gốc: Sau khi đọc bài viết, câu trả lời đúng nhất là: C: không nên vứt túi đựng đồ ăn trưa. Bài báo đề cập rằng các câu lạc bộ môi trường yêu cầu học sinh mang bữa trưa của mình trong túi để có thể sử dụng lại, như một phần trong nỗ lực làm cho môi trường trong sạch.
- Phản hổi được viết lại: Dựa vào nguồn tài liệu được cung cấp, câu trả lời đúng nhất là: C: không nên vứt túi đựng đồ ăn trưa. Tài liệu đề cập rằng các câu lạc bộ môi trường yêu cầu học sinh mang bữa trưa của mình trong túi để có thể sử dụng lại, như một phần trong nỗ l

In [108]:
pprint(a)

('{"rewritten_prompt": "Dựa vào nguồn tài liệu được cung cấp, đối với vụ án '
 'Kathryn Smith và Matthew Rigby liên quan đến cáo buộc giết con gái Ayeeshia '
 'Jane Smith, cả hai cha mẹ đều phủ nhận tội ác. Tài liệu bàn luận về tiền sử '
 'tự làm hại bản thân, lạm dụng chất gây nghiện, và vấn đề sức khỏe tâm thần '
 'của Kathryn Smith, cũng như việc cô mang thai ngoài ý muốn. Nó cũng đề cập '
 'đến sự tham gia của các dịch vụ xã hội, những người đã phát hiện ra Smith '
 'sống trong điều kiện không phù hợp và sau đó lo ngại về khả năng chăm sóc '
 'con của cô ấy do các vấn đề cá nhân của cô ấy. Phiên tòa đang diễn ra, và cả '
 'hai bị cáo cũng không nhận tội cho hành vi gây ra cái chết và hành vi tàn ác '
 'đối với trẻ em."}')


In [ ]:
text = 'Trong nhiệm vụ này, bạn được yêu cầu xác định xem có ít phím đàn hoặc quãng tám hơn trong bối cảnh một đoạn văn mô tả đàn mandolin Lombardi hay không. Đoạn văn được cung cấp đưa ra các đặc điểm cụ thể của đàn mandolin Lombardi và nó khác với đàn mandolin Neapolitan như thế nào. Để trả lời câu hỏi chúng ta có thể làm theo các bước sau:\n\n1. Xác định thông tin liên quan trong đoạn văn về phím đàn và quãng tám trong đàn mandolin Lombardi. 2. So sánh số phím đàn với số quãng tám. Trong đoạn văn, nó nói rằng đàn mandolin Lombardi có "20 phím đàn, bao gồm ba quãng tám, cộng thêm 5 nốt." Bây giờ hãy so sánh số phím đàn với số quãng tám:\n- Phím đàn: 20\n- Quãng tám: 3\n\nTừ sự so sánh này, chúng ta có thể thấy rằng có nhiều phím đàn (20) hơn quãng tám (3). Vì vậy, câu trả lời cho câu hỏi "Có ít phím đàn hoặc quãng tám hơn không?" là: Có ít quãng tám hơn.'